In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint


In [2]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("TSM")
qcom = yf.Ticker("QCOM")

tsm_data = tsm.history(period="60d", interval="30m")
qcom_data = qcom.history(period="60d", interval="30m")

In [3]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [4]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-02-21 09:30:00-05:00  124.430000  124.500000  122.910004  124.209999   
2024-02-21 10:00:00-05:00  124.199997  124.599998  123.570000  123.584999   
2024-02-21 10:30:00-05:00  123.599998  123.629997  123.139999  123.430000   
2024-02-21 11:00:00-05:00  123.419998  124.589996  123.360001  124.339996   
2024-02-21 11:30:00-05:00  124.349998  124.779999  124.014603  124.769997   
...                               ...         ...         ...         ...   
2024-05-15 13:30:00-04:00  155.410004  155.839996  155.369995  155.769897   
2024-05-15 14:00:00-04:00  155.759995  156.000000  155.419998  155.550003   
2024-05-15 14:30:00-04:00  155.559998  155.720001  155.389999  155.619995   
2024-05-15 15:00:00-04:00  155.610001  155.740005  155.262802  155.675003   
2024-05-15 15:30:00-04:00  155.679993  155.770004  154.779999  155.550003   

In [5]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [6]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    732
 1.0     24
-1.0     23
Name: count, dtype: int64


In [7]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-553.7629928588867


In [8]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

0.8259192738119142

In [9]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [10]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2024-02-21 09:30:00-05:00    8.447266e-08
2024-02-21 10:00:00-05:00   -5.506699e-01
2024-02-21 10:30:00-05:00   -3.092317e-01
2024-02-21 11:00:00-05:00    2.951783e-01
2024-02-21 11:30:00-05:00    2.461446e-01
                                 ...     
2024-05-15 13:30:00-04:00   -5.689066e+00
2024-05-15 14:00:00-04:00   -5.508389e+00
2024-05-15 14:30:00-04:00   -5.178230e+00
2024-05-15 15:00:00-04:00   -5.139744e+00
2024-05-15 15:30:00-04:00   -5.256477e+00
Length: 780, dtype: float64

In [11]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [12]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [13]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [14]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

In [15]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], 1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1]
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = -tsm_qcom_dif['signal']
qcom_data['signal'] = tsm_qcom_dif['signal'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    759
 1.0     10
-1.0     10
Name: count, dtype: int64


TSMC Profit =>  9.786697387695312
QCOM Profit =>  -18.744493999999804
Pair profit =>  -8.957796612304492


In [16]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [17]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']

In [18]:
# calculate cointegration between the two tickers
cointegration = coint(tsm_data['Close'], qcom_data['Close_NORM'])
print(cointegration)



(-2.9494605085794454, 0.12271279157926962, array([-3.91055417, -3.34398476, -3.0498989 ]))


In [19]:
# Calculate the z score of the difference
tsm_qcom_dif['z_score'] = (tsm_qcom_dif['Close'] - tsm_qcom_dif['Close'].rolling(window=20).mean()) / tsm_qcom_dif['Close'].rolling(window=20).std()

In [20]:
# function to implement a flag that is (-1 sell order open, 0 no order open, 1 buy order open) to control the signal_z_score signals

def restrict_signals(dataset, signal_col, flag_col):
    dataset[flag_col] = 0
    # cicle to put flags
    for i in range(dataset.shape[0]):
        if dataset[signal_col].iloc[i] == 1:
            if dataset[flag_col].iloc[i] < 1:
                dataset[flag_col].iloc[i] += 1
                print('Buy signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset buy signal')
        elif dataset[signal_col].iloc[i] == -1:
            if dataset[flag_col].iloc[i] > -1:
                dataset[flag_col].iloc[i] += -1
                print('Sell signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset sell signal')

    return dataset
        

In [21]:
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
tsm_qcom_dif['flag_signal_z_score'] = 0
tsm_qcom_dif['flag_signal_z_score'] = tsm_qcom_dif['flag_signal_z_score'].astype(int)
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
tsm_qcom_dif['flag_signal_z_score'] = 0
tsm_qcom_dif['flag_signal_z_score'] = tsm_qcom_dif['flag_signal_z_score'].astype(int)

for i in range(tsm_qcom_dif.shape[0]):
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        # Check the actual boolean value for the current row
        if not tsm_qcom_dif.loc[i, 'flag_signal_z_score']:  # Use `.loc` for efficient indexing
            tsm_qcom_dif.loc[i:, 'flag_signal_z_score'] += 1  # Update from current row onwards
            print('Buy signal')
        else:
            tsm_qcom_dif.loc[i, 'signal_z_score'] = 0  # Reset signal for current row only
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        # Similar logic for sell signal
        if tsm_qcom_dif.loc[i, 'flag_signal_z_score'] > 0:
            tsm_qcom_dif.loc[i, 'flag_signal_z_score'] += -1
            print('Sell signal')
        else:
            tsm_qcom_dif.loc[i, 'signal_z_score'] = 0
            print('Reset sell signal')

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()
for i in range(tsm_qcom_dif.shape[0]):
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        # Check the actual boolean value for the current row
        if not tsm_qcom_dif.loc[i, 'flag_signal_z_score']:  # Use `.loc` for efficient indexing
            tsm_qcom_dif.loc[i:, 'flag_signal_z_score'] += 1  # Update from current row onwards
            print('Buy signal')
        else:
            tsm_qcom_dif.loc[i, 'signal_z_score'] = 0  # Reset signal for current row only
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        # Similar logic for sell signal
        if tsm_qcom_dif.loc[i, 'flag_signal_z_score'] > 0:
            tsm_qcom_dif.loc[i, 'flag_signal_z_score'] += -1
            print('Sell signal')
        else:
            tsm_qcom_dif.loc[i, 'signal_z_score'] = 0
            print('Reset sell signal')

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

KeyError: 52

## Calculate Profit using Z-score

In [ ]:
# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score.index, y=buy_signals_z_score['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score.index, y=sell_signals_z_score['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # 1 buy || - 1 sell

tsm_data['signal_z_score'] = tsm_qcom_dif['signal_z_score']
qcom_data['signal_z_score'] = - tsm_qcom_dif['signal_z_score'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


In [ ]:
#Testar Métodos de Mercados Diferentes, Mercados Tecnológicos
# A cointegração muda com 